<a href="https://colab.research.google.com/github/jenn2i/AI_sec/blob/main/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q autoviz
!pip install -q -U --pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip -q install transformers
!pip -q install torch

In [ ]:
pip install tld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
pip install whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install dnspython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import re
import tld
from tld import get_tld
import whois
import dns.resolver

In [ ]:
data = pd.read_csv('/content/malicious_phish.csv')

In [ ]:
data

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [ ]:
# 악성인 데이터만 뽑아냄
df = data.loc[(data['type']=="defacement") | (data['type']=="phishing") | (data['type']=="malware")].copy()

In [ ]:
df['url'].value_counts()

http://style.org.hc360.com/css/detail/mysite/siteconfig/pro_control.css                                                                                               180
http://www.pubyun.com/wiki/lib/exe/css.php?t=dokuwiki&tseed=f1f16e8c6142fab8553f2b9cfe045961                                                                          146
http://9779.info/%E6%A0%91%E5%8F%B6%E8%B4%B4%E7%94%BB%E4%BD%9C%E5%93%81/                                                                                               91
http://9779.info/%E6%A0%91%E5%8F%B6%E7%B2%98%E8%B4%B4%E7%94%BB/                                                                                                        88
http://www.js182.com/app/member/SportOption.php?uid=guest&langx=gb                                                                                                     79
                                                                                                                                                      

In [ ]:
#url 기반 특징
#url 길이
df['url_len'] = df['url'].apply(lambda x: len(str(x)))

In [ ]:
# domain 여부, 길이
def try_get_tld(x):
    try:
        return tld.get_tld(x, as_object=True, fail_silently=False, fix_protocol=True)
    except tld.exceptions.TldBadUrl:
        return None
    except tld.exceptions.TldDomainNotFound:
        return None
    except Exception as e:
        return None

res = df['url'].apply(lambda x: try_get_tld(x))
df['domain'] = res.apply(lambda x: x.domain if x else None)
df['domain_len'] = res.apply(lambda x: len(x.domain) if x else 0)

In [ ]:
# path 여부, 길이
df['path'] = res.apply(lambda x: x.parsed_url.path if x else None)
df['path_len'] = res.apply(lambda x: len(x.parsed_url.path) if x else 0)

In [ ]:
# TLD 여부, 길이
df['TLD'] = res.apply(lambda x: x.tld if x else None)
df['TLD_len'] = res.apply(lambda x: len(x.tld) if x else 0)

In [ ]:
# parameter 여부, 길이
df['parameter'] = res.apply(lambda x: x.parsed_url.query if x else None)
df['parameter_len'] = res.apply(lambda x: len(x.parsed_url.query) if x else 0)

In [ ]:
#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

df['fd_length'] = df['url'].apply(lambda i: fd_length(i))

In [ ]:
#url에 특수기호 포함
feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
for a in feature:
    df[a] = df['url'].apply(lambda i: i.count(a))

In [ ]:
#url 내 숫자, 글자 개수
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

df['num_count'] = df['url'].apply(lambda x: digit_count(x))
df['letter_count'] = df['url'].apply(lambda x: letter_count(x))

In [ ]:
pip install res

  Using cached res-0.1.7.tar.gz (3.8 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#서브 도메인 개수
def having_Sub_Domain(result):
    if result is not None:
        domain = result.domain
        subdomain = result.subdomain
        if subdomain == "":
            return 0
        dot = subdomain.count('.')
        if dot == 0:
            return 1
        else:
            return 1
    else:
        return 0
    #res가 존재하지 않아도 그냥 -1
df['domain'] = res.apply(lambda x: x.domain if x else None)
df['sub_domain'] = res.apply(lambda x: having_Sub_Domain(x))

In [ ]:
pip install tld

In [ ]:
#서브도메인개수2

from tld import get_tld

# URL이 IP 주소인지 점검 (having_IP_Address)
# URL이 IP주소이면 -1

# -1 : 정상 / 0 : 의심 / 1 : 피싱
def having_Sub_Domain(url) :
    url = remove_www(url)
    domain = get_tld(url, as_object=True)
    if domain.subdomain == "" :
        return -1
    dot = domain.subdomain.count('.')
    if dot == 0 :
        return 0
    else :
        return 1

# URL에서 "www."을 제거하고 반환
def remove_www(url) :
    if "www." in url[:12] :
        url = url.replace("www.", "")
    return url

In [ ]:
#shrotening service 포함 여부
df['shortening_service'] = df['url'].apply(lambda x: 1 if re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net', x) else 0)

In [ ]:
#정상 IP주소 여부
import re
def has_ip_address(url):
    ipv4_pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
    ipv6_pattern = r'([a-fA-F0-9]{1,4}::?)+'

    if re.search(ipv4_pattern, url) or re.search(ipv6_pattern, url):
        return 1
    else:
        return 0

df['having_ip_address'] = df['url'].apply(has_ip_address)

In [ ]:
#http, https 여부
df['Protocal'] = df['url'].apply(lambda x: 1 if (urlparse(x).scheme == "http")|(urlparse(x).scheme == "https") else 0)

In [ ]:
#url 단축
import requests

def Shortening_Service(url) :
    response = requests.get(url)
    # redirect detection
    for resp in response.history :
        if resp.status_code == 301 or resp.status_code == 302 :
            return 1
    else :
        return 0

In [ ]:
#도메인 등록 기간
import whois, sys
import urllib.parse
from datetime import datetime

def Domain_registration_length(url) :
    try :
        total_date = get_total_date(url)
        if total_date <= 365 :
            return 1
        else :
            return 0
    except whois.parser.PywhoisError :
        return 1

# domain 유효 기간 계산
def get_total_date(url) :
    domain = whois.whois(url)
#     print(domain)
    if type(domain.expiration_date) is list :
        expiration_date = domain.expiration_date[0]
    else :
        expiration_date = domain.expiration_date

    if type(domain.updated_date) is list :
        updated_date = domain.updated_date[0]
    else :
        updated_date = domain.updated_date

    total_date = (expiration_date - updated_date).days

    return total_date

In [ ]:
#도메인 내 IP 주소 탐지
import urllib.parse

def having_IP_Address(url) :
    parse = urllib.parse.urlparse(url)
    domain = parse.netloc.split('.')
    try :
        int(domain[-1], 0)
        return 1
    except ValueError :
        return 0

In [ ]:
#html 및 javascript 기반 특징
#html 소스코드 내 도메인 존재 여부

def try_get_tld(x):
    try:
        return tld.get_tld(x, as_object=True, fail_silently=False, fix_protocol=True)
    except tld.exceptions.TldBadUrl:
        return None
    except tld.exceptions.TldDomainNotFound:
        return None
    except Exception as e:
        return None

res = df['url'].apply(lambda x: try_get_tld(x))
df['domain'] = res.apply(lambda x: x.domain if x else None)
df['domain_len'] = res.apply(lambda x: len(x.domain) if x else 0)

In [ ]:
#html 소스코드 길이

import requests

def get_html_source_length(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            html_length = len(response.content)
            return html_length
        else:
            print("Error: Failed to fetch URL, status code:", response.status_code)
            return None
    except Exception as e:
        print("Error:", e)
        return None

def is_malicious_url(url):
    html_source_length = get_html_source_length(url)
    if html_source_length is not None:
        if html_source_length < 5000:
            return 1  # HTML 소스 코드 길이가 5000보다 작으면 피싱으로 분류
        elif html_source_length < 50000:
            return 1  # HTML 소스 코드 길이가 5000 이상이고 50000보다 작으면 의심으로 분류
        else:
            return 0

In [ ]:
#피싱 도메인 및 ip 사용
import re
def has_ip_address(url):
    ipv4_pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
    ipv6_pattern = r'([a-fA-F0-9]{1,4}::?)+'

    if re.search(ipv4_pattern, url) or re.search(ipv6_pattern, url):
        return 1
    else:
        return 0

df['having_ip_address'] = df['url'].apply(has_ip_address)

In [ ]:
#웹사이트 트래픽
import requests
from bs4 import BeautifulSoup

def get_alexa_rank(url):
    try:
        # Alexa에서 URL의 순위를 확인
        alexa_url = f"https://www.alexa.com/siteinfo/{url}"
        response = requests.get(alexa_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        rank_container = soup.find('div', class_='rank-row')
        rank = rank_container.find('div', class_='rank-global').text.strip()
        return int(rank.replace(',', ''))  # 숫자에서 쉼표 제거하고 정수로 변환
    except Exception as e:
        print("Error:", e)
        return None

def is_malicious_url(url):
    try:
        rank = get_alexa_rank(url)
        if rank is not None and rank <= 100000:
            return 0
        else:
            return 1
    except Exception as e:
        print("Error:", e)
        return False

In [ ]:
#브라우저 아이콘

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse

def Favicon(url):
    tag_link = get_favicon_tag(url)

    if not tag_link :
        return 0

    for link in tag_link:
#         print(link.get('href'))
        parse = urlparse(link.get('href'))

        if parse.netloc == '':
            return 0
        else:
            return 1

def get_favicon_tag(url):
    resp = urlopen(url)
    soup = BeautifulSoup(resp, 'html.parser')

    tag_link = soup.findAll("link", rel="shortcut icon")
    return tag_link

In [ ]:
#port 스캔
import socket

# HTTPS 검사, domain으로 받기
def port_scan(domain) :
    try :
        ip = socket.gethostbyname(domain)
    except :
        return 1

    socket.setdefaulttimeout(2)

    ports = [80, 21, 22, 23, 445, 1433, 1521, 3306, 3389]
    for port in ports :
        s = socket.socket()
        if port == 80 :
            try :
                s.connect((ip, port))
#                 print("%d port is open!" % port)
                s.close()
            except :
#                 print("%d port is closed!" % port)
                return -1
        else :
            try :
                s.connect((ip, port))
#                 print("%d port is open!" % port)
                s.close()
                return 1
            except :
#                 print("%d port is closed!" % port)
                pass

    return 0

In [ ]:
#파일 확장자 여부 (js, txt, exe)
df['path_extension']=res.apply(lambda x: x.parsed_url.path.rsplit(".",1)[-1] if x else "0")
df['filename_extension'] = df['path_extension'].apply(lambda x: 1 if "js" in x or "exe" in x or "txt" in x  else 0)

In [ ]:
#abnormal url 추출
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 0
    else:
        # print 'No matching pattern found'
        return 1

df['abnormal_url'] = df['url'].apply(lambda x: abnormal_url(x))

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# url 종류 라벨인코딩
le = LabelEncoder()
df['class_url'] = le.fit_transform(df['type'])

In [ ]:
df.head()

,url,type,url_len,domain,domain_len,path,path_len,TLD,TLD_len,parameter,...,num_count,letter_count,sub_domain,shortening_service,having_ip_address,Protocal,path_extension,filename_extension,abnormal_url,class_url
0,br-icloud.com.br,phishing,16,br-icloud,9,,0,com.br,6,,...,0,13,0,0,0,0,,0,1,2
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,garage-pirenne,14,/index.php,10,be,2,option=com_content&view=article&id=70&vsig70_0=15,...,7,63,1,0,0,1,php,0,0,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,adventure-nicaragua,19,/index.php,10,net,3,option=com_mailto&tmpl=component&link=aHR0cDov...,...,22,199,0,0,0,1,php,0,0,0
8,http://www.pashminaonline.com/pure-pashminas,defacement,44,pashminaonline,14,/pure-pashminas,15,com,3,,...,0,37,1,0,0,1,/pure-pashminas,0,0,0
11,http://www.ikenmijnkunst.nl/index.php/expositi...,defacement,64,ikenmijnkunst,13,/index.php/exposities/exposities-2006,37,nl,2,,...,4,50,1,0,0,1,php/exposities/exposities-2006,0,0,0


In [ ]:
df.columns

Index(['url', 'type', 'url_len', 'domain', 'domain_len', 'path', 'path_len',
       'TLD', 'TLD_len', 'parameter', 'parameter_len', 'fd_length', '@', '?',
       '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '//', 'num_count',
       'letter_count', 'sub_domain', 'shortening_service', 'having_ip_address',
       'Protocal', 'path_extension', 'filename_extension', 'abnormal_url',
       'class_url'],
      dtype='object')

In [ ]:
X = df[['url_len', 'domain_len', 'path_len','TLD_len','parameter_len', 'fd_length',
        '@', '?', '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '//',
        'num_count','letter_count', 'shortening_service', 'having_ip_address', 'Protocal',
       'filename_extension', 'sub_domain','abnormal_url']]
y = df['class_url']

In [ ]:
#데이터분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024, stratify=y)

In [ ]:
#학습(랜덤포레스트 예시)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
#예측
pred = model.predict(X_test)
pred1 = model.predict_proba(X_test)

In [ ]:
#평가
from sklearn.metrics import accuracy_score, f1_score
y_score = accuracy_score(y_test, pred)
macro_f1 = f1_score(y_test, pred, average="macro")
f1 = f1_score(y_test, pred, average=None)

In [ ]:
macro_f1

0.9752579095099504

In [ ]:
y_score

0.9767582589986105

In [ ]:
f1

array([0.9819051 , 0.97021944, 0.97364919])

In [ ]:
import numpy as np

# feature_importance를 numpy 배열로 변환
feature_importance = np.array(model.feature_importances_)

# 특성 중요도를 내림차순으로 정렬한 후의 인덱스를 얻음
sorted_indices = np.argsort(feature_importance)[::-1]

# 각 특성의 중요도를 출력 (내림차순)
for i, index in enumerate(sorted_indices):
    importance = feature_importance[index]
    print(f"Feature {index+1}: Importance = {importance}")

#27 > 24 > 6

Feature 27: Importance = 0.13518730373547316
Feature 24: Importance = 0.12143951072557625
Feature 6: Importance = 0.09000182293634537
Feature 2: Importance = 0.07028255330925494
Feature 3: Importance = 0.07014892594155195
Feature 20: Importance = 0.06766289894699141
Feature 21: Importance = 0.06382186272105939
Feature 19: Importance = 0.06333972504278845
Feature 1: Importance = 0.059945038507064875
Feature 4: Importance = 0.048978279737244126
Feature 10: Importance = 0.03628662592223214
Feature 11: Importance = 0.03373286211142201
Feature 5: Importance = 0.03356686766329797
Feature 9: Importance = 0.028313030059762198
Feature 25: Importance = 0.027246773466951354
Feature 26: Importance = 0.013033607601614842
Feature 13: Importance = 0.01101106473206629
Feature 8: Importance = 0.010039948472037909
Feature 23: Importance = 0.008103469863848762
Feature 22: Importance = 0.003559237917849638
Feature 7: Importance = 0.002294520084029648
Feature 18: Importance = 0.0008801560403725938
Feature 